
![Py4Eng](img/logo.png)

# Object Oriented Programming
## Yoav Ram

# Introduction to OOP


*Everything in Python is an object* so everything in Python has a class. 
You can find out which class an object belongs to by using the default property `__class__` or giving it to the function `type`. Even the return of the `type` function has a class - it's of class `type`!

In [2]:
type(5)

int

In [3]:
a = 'Hi'
print(a.__class__)
print(type(a.__class__))

<class 'str'>
<class 'type'>


The basic object type is `object`:

In [3]:
print(object)
print(type(object))

<class 'object'>
<class 'type'>


Every class implicitly inherits from `object`, so all classes have some default attributes:

In [14]:
object_dir = set(dir(object))
int_dir = set(dir(int))

print("object's attributes:\n", object_dir)
print("Attributes in object and in int:\n", object_dir & int_dir)
print("Attributes in object but not in int:\n", object_dir - int_dir)

object's attributes:
 {'__delattr__', '__setattr__', '__gt__', '__reduce_ex__', '__reduce__', '__format__', '__hash__', '__le__', '__ge__', '__dir__', '__init__', '__class__', '__str__', '__repr__', '__sizeof__', '__lt__', '__ne__', '__doc__', '__new__', '__eq__', '__subclasshook__', '__getattribute__'}
Attributes in object and in int:
 {'__delattr__', '__setattr__', '__gt__', '__reduce_ex__', '__reduce__', '__format__', '__hash__', '__le__', '__ge__', '__dir__', '__init__', '__class__', '__str__', '__repr__', '__sizeof__', '__lt__', '__ne__', '__doc__', '__new__', '__eq__', '__subclasshook__', '__getattribute__'}
Attributes in object but not in int:
 set()


## Class definition

In [4]:
class MyClass:
    pass

We defined a new class called `MyClass`.
We can now create a new instance of our new class:

In [5]:
my_class1 = MyClass()
my_class2 = MyClass()
print(id(my_class1))
print(type(my_class1))
print(id(my_class2))
print(type(my_class2))

73184928
<class '__main__.MyClass'>
73187168
<class '__main__.MyClass'>


Note that both instances have the same type but a different `id`, so they are not the same object:

In [6]:
my_class1 is my_class2

False

We can check if an object is an instance of a class with the `isinstance` function:

In [7]:
print(isinstance(my_class1, MyClass))
print(isinstance(my_class1, object))
print(isinstance(my_class1, int))

True
True
False


Because all classes have a `__doc__` attribute inherited from `object`, 
you can pass any instance to `help`:

In [9]:
help(my_class1) # uses the __doc__ attribute

Help on MyClass in module __main__ object:

class MyClass(builtins.object)
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



The same goes for `type` etc.

# `Point` class

Let's learn more by writing a class for a point in a two dimensional Euclidian space ($\mathbb{R}^2$).

We start with the class definition (`class`) and the constructor (`__init__`) which initialized the attributes of the class instance.

Note:

* The first argument to methods (member functions) is always `self`, a reference to the instance.
* We *assert* that the `__init__` arguments are real numbers using the `isinstance` function.

In [16]:
class Point:
    """A point (x,y) in the Euclidian plane.
    """
    
    def __init__(self, x, y):        
        # assert isinstance(x, (int, float)) and isinstance(y, (int, float))
        self.x = float(x)
        self.y = float(y)

In [17]:
origin = Point(0, 0)
print("origin", origin.x, origin.y)

p = Point(1, 2)
print("point", p.x, p.y)

origin 0.0 0.0
point 1.0 2.0


I left the `assert isinstance...` line in a comment because we don't need it - `float(x)` and `float(y)` will raise an exception, and the correct one (not `AssertionError`).

Notice that when we send a `Point` to the console we get:

In [18]:
p

Which is not useful, so we will define how `Point` is represented in the console by implementing `__repr__` and/or which must return a string:

In [36]:
class Point:
    """A point (x,y) in the Euclidian plane.
    """
    
    def __init__(self, x, y):        
        self.x = float(x)
        self.y = float(y)
        
    def __repr__(self):
        # what a programmer wants to see, should work with eval
        return "{}({!r}, {!r})".format(type(self).__name__, self.x, self.y)
    
    def __str__(self):
        # what a user wants to see
        return "({!r}, {!r})".format(self.x, self.y)

In [37]:
p=Point(1,2)
p

Point(1.0, 2.0)

In [38]:
print(p)

(1.0, 2.0)


It would also be nice to be able to do `x, y = p` rather than `x, y = p.x, p.y`. To do that, we implement the `__iter__` method, which should either return a generator or be a generator function:

In [45]:
class Point:
    """A point (x,y) in the Euclidian plane.
    """
    
    def __init__(self, x, y):        
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        #return (z for z in (self.x, self.y)) # this creates a tuple
        yield self.x
        yield self.y
    
    def __repr__(self):
        # what a programmer wants to see, should work with eval
        return "{}({!r}, {!r})".format(type(self).__name__, *self)

In [46]:
p = Point(1.5, 5.6)
x, y = p
print(x, y, p)

1.5 5.6 Point(1.5, 5.6)


Next up we define a method to add two points. Addition is by elements: $(x_1, y_1) + (x_2, y_2) = (x_1+x_2, y_1+y_2)$.

We also allow to add an `int` or `float`, in which case we add the point to a another point with both coordinates equal to the argument value.

In [52]:
class Point:
    """A point (x,y) in the Euclidian plane.
    """
    
    def __init__(self, x, y):        
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        yield self.x
        yield self.y
    
    def __repr__(self):
        return "{}({!r}, {!r})".format(type(self).__name__, *self)
    
    def add(self, other):
        if isinstance(other, numbers.Real):
            other = Point(other, other)
        if isinstance(other, type(self)):
            return Point(*(z1 + z2 for z1,z2 in zip(self, other)))
        else:
            raise TypeError(
                "unsupported operand type(s) for +: '{}' and '{}'".format(
                    type(self).__name__, type(other).__name__
            ))

In [53]:
Point(1,1).add(Point(2,2))

Point(3.0, 3.0)

In [54]:
Point(1,1).add(2)

Point(3.0, 3.0)

In [55]:
Point(1,1).add('2')

TypeError: unsupported operand type(s) for +: 'Point' and 'str'

A nicer way to implement addition is to **overload** the addition operator `+` by implementing `__add__`, which is a name Python reserves for addition operations (those are double underscores). In that case, we don't need to raise the `TypeError` exception but rather just return the `NotImplemented` constant and Python will take care of the error (after checking if the right hand object knows how to do right-add using `__radd__`):

In [56]:
class Point:
    """A point (x,y) in the Euclidian plane.
    """
    
    def __init__(self, x, y):        
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        yield self.x
        yield self.y
    
    def __repr__(self):
        return "{}({!r}, {!r})".format(type(self).__name__, *self)
    
    def __add__(self, other):
        if isinstance(other, numbers.Real):
            other = Point(other, other)
        if isinstance(other, type(self)):
            return Point(*(z1 + z2 for z1,z2 in zip(self, other)))
        return NotImplemented

In [57]:
Point(1,1) + Point(2,2)

Point(3.0, 3.0)

In [58]:
Point(1,1) + 2

Point(3.0, 3.0)

In [59]:
Point(1, 1) + '2'

TypeError: unsupported operand type(s) for +: 'Point' and 'str'

We want to be a able to compare `Point`s:

In [55]:
Point(1,2) == Point(2,1)

False

In [56]:
Point(1,2) == Point(1,2)

False

In [57]:
p = Point(0, 0)
p == p

True

In [58]:
Point(1,2) > Point(2,1)

TypeError: unorderable types: Point() > Point()

So the default `==` checks by identity and `>` is not defined. 

We should overload both these operators by implementing `__eq__` and `__gt__`:

In [61]:
class Point:
    """A point (x,y) in the Euclidian plane.
    """
    
    def __init__(self, x, y):        
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        yield self.x
        yield self.y
    
    def __repr__(self):
        return "{}({!r}, {!r})".format(type(self).__name__, *self)
    
    def __add__(self, other):
        if isinstance(other, numbers.Real):
            other = Point(other, other)
        if isinstance(other, type(self)):
            return Point(*(z1 + z2 for z1,z2 in zip(self, other)))
        return NotImplemented
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __gt__(self, other):
        return all(z1 > z2 for z1,z2 in zip(self, other))

First we check if two points are equal:

In [70]:
Point(1,0) == Point(1,2)

False

In [71]:
Point(1,0) == Point(1,0)

True

Then if one is *strictly* smaller than the other:

In [72]:
Point(1,0) > Point(1,2)

False

In [73]:
Point(5,6) > Point(1,2)

True

Note that by implementing `==` and `>` we have logically defined all other comparison operators. 
Indeed, Python fills in the other comparison operators:

In [74]:
Point(5,6) < Point(1,4)

False

The addition operator `+` returns a **new instance**.

In [79]:
p = Point(1, 1)
pp = p
p += Point(1,2)
print(p)
print(pp)

Point(2.0, 3.0)
Point(2.0, 3.0)


Next we will write a method that implements `+=` efficiently, so that instead of returning a new instance, in changes the current instance **in-place**. This is called an [augmented arithmetic assignments](https://docs.python.org/3.5/reference/datamodel.html?highlight=__iadd__#object.__iadd__). These methods should return their result.

In [66]:
class Point:
    """A point (x,y) in the Euclidian plane.
    """
    
    def __init__(self, x, y):        
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        yield self.x
        yield self.y
    
    def __repr__(self):
        return "{}({!r}, {!r})".format(type(self).__name__, *self)
    
    def __add__(self, other):
        if isinstance(other, numbers.Real):
            other = Point(other, other)
        if isinstance(other, type(self)):
            return Point(*(z1 + z2 for z1,z2 in zip(self, other)))
        return NotImplemented
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __gt__(self, other):
        return all(z1 > z2 for z1,z2 in zip(self, other))
    
    def __iadd__(self, other):
        if isinstance(other, numbers.Real):
            other = Point(other, other)
        if isinstance(other, type(self)):
            self.x += other.x
            self.y += other.y
            return self
        return NotImplemented

In [67]:
p = Point(1, 1)
pp = p
p + Point(1,2)
print(p)
p += Point(1,2)
print(p)
print(pp)

Point(1.0, 1.0)
Point(2.0, 3.0)
Point(2.0, 3.0)


We now write a method that given many points, checks if the current point is more extreme than the other points.

**Note:** specifying a function argument with a `*` before its name says that we can give zero or more values and they will be packed in a `tuple`.

In [64]:
class Point:
    """A point (x,y) in the Euclidian plane.
    """
    
    def __init__(self, x, y):        
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        yield self.x
        yield self.y
    
    def __repr__(self):
        return "{}({!r}, {!r})".format(type(self).__name__, *self)
    
    def __add__(self, other):
        if isinstance(other, numbers.Real):
            other = Point(other, other)
        if isinstance(other, type(self)):
            return Point(*(z1 + z2 for z1,z2 in zip(self, other)))
        return NotImplemented
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __gt__(self, other):
        return all(z1 > z2 for z1,z2 in zip(self, other))
    
    def __iadd__(self, other):
        if isinstance(other, numbers.Real):
            other = Point(other, other)
        if isinstance(other, type(self)):
            self.x += other.x
            self.y += other.y
            return self
        return NotImplemented
    
    def is_extreme(self, *points):
        return all(self > point for point in points)

In [65]:
p = Point(5, 6)
p.is_extreme(Point(1,1))

True

In [84]:
p.is_extreme(Point(1,1), Point(2,5), Point(6,2))

False

We can also use the method via the class instead of the instance, and give the instance of interest (the one that we want to know if it is the extreme) as the first argument `self`. Similarly, we can either do `'hi'.upper()` or `str.upper('hi')`.

In [85]:
Point.is_extreme(Point(7,8), Point(1,1), Point(4,5), Point(2,3))

True

Let's add a method to return a polar representation of `Point`. We'll need to calculate the magnitude, which we might as well do with the `__abs__` method (corresponding to `abs`) and an `angle` method:

In [62]:
import math

class Point:
    """A point (x,y) in the Euclidian plane.
    """
    
    def __init__(self, x, y):        
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        yield self.x
        yield self.y
    
    def __repr__(self):
        return "{}({!r}, {!r})".format(type(self).__name__, *self)
    
    def __add__(self, other):
        if isinstance(other, numbers.Real):
            other = Point(other, other)
        if isinstance(other, type(self)):
            return Point(*(z1 + z2 for z1,z2 in zip(self, other)))
        return NotImplemented
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __gt__(self, other):
        return all(z1 > z2 for z1,z2 in zip(self, other))
    
    def __iadd__(self, other):
        if isinstance(other, numbers.Real):
            other = Point(other, other)
        if isinstance(other, type(self)):
            self.x += other.x
            self.y += other.y
            return self
        return NotImplemented
    
    def is_extreme(self, *points):
        return all(self > point for point in points)
    
    def __abs__(self):
        return math.hypot(*self)
    
    def angle(self):
        return math.atan2(self.y ,self.x)
    
    def to_polar(self):
        return abs(self), self.angle()

In [63]:
print(Point(1, 0).to_polar())
print(Point(-1, 0).to_polar())
print(Point(1, 1).to_polar())
print(Point(2, 2).to_polar())

(1.0, 0.0)
(1.0, 3.141592653589793)
(1.4142135623730951, 0.7853981633974483)
(2.8284271247461903, 0.7853981633974483)


If we have `topolar` that returns a tuple of polar coordinates, we might want to have `frompolar` that creates a new instance from a tuple of polar coordinates. This is called an **alternative constructor** and must be implemented as a **class method** as we don't have an instance at the time we call the method:

In [124]:
import math

class Point:
    """A point (x,y) in the Euclidian plane.
    """
    
    def __init__(self, x, y):        
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        yield self.x
        yield self.y
    
    def __repr__(self):
        return "{}({!r}, {!r})".format(type(self).__name__, *self)
    
    def __add__(self, other):
        if isinstance(other, numbers.Real):
            other = Point(other, other)
        if isinstance(other, type(self)):
            return Point(*(z1 + z2 for z1,z2 in zip(self, other)))
        return NotImplemented
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __gt__(self, other):
        return all(z1 > z2 for z1,z2 in zip(self, other))
    
    def __iadd__(self, other):
        if isinstance(other, numbers.Real):
            other = Point(other, other)
        if isinstance(other, type(self)):
            self.x += other.x
            self.y += other.y
            return self
        return NotImplemented
    
    def is_extreme(self, *points):
        return all(self > point for point in points)
    
    def __abs__(self):
        return math.hypot(*self)
    
    def angle(self):
        return math.atan2(self.y ,self.x)
    
    def to_polar(self):
        return abs(self), self.angle()
    
    @classmethod
    def from_polar(cls, radius, angle):
        x = radius * math.cos(angle)
        y = radius * math.sin(angle)
        return Point(x, y)

In [127]:
p1 = Point(3, 4.5)
r, a = p1.to_polar()
p2 = Point.from_polar(r, a)
p1, p2, p1 == p2

(Point(3.0, 4.5), Point(3.0000000000000004, 4.5), False)

We got a `False` because there is some numerical error.

In [131]:
p1 = Point(1, 1)
r, a = p1.to_polar()
p2 = Point.from_polar(r * 2, a)
p1, p2

(Point(1.0, 1.0), Point(2.0000000000000004, 2.0000000000000004))

We can also implement `__complex__` to allow to convert our `Point` to a complex number:

In [13]:
import math

class Point:
    """A point (x,y) in the Euclidian plane.
    """
    
    def __init__(self, x, y):        
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        yield self.x
        yield self.y
    
    def __repr__(self):
        return "{}({!r}, {!r})".format(type(self).__name__, *self)
    
    def __add__(self, other):
        if isinstance(other, numbers.Real):
            other = Point(other, other)
        if isinstance(other, type(self)):
            return Point(*(z1 + z2 for z1,z2 in zip(self, other)))
        return NotImplemented
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __gt__(self, other):
        return all(z1 > z2 for z1,z2 in zip(self, other))
    
    def __iadd__(self, other):
        if isinstance(other, numbers.Real):
            other = Point(other, other)
        if isinstance(other, type(self)):
            self.x += other.x
            self.y += other.y
            return self
        return NotImplemented
    
    def is_extreme(self, *points):
        return all(self > point for point in points)
    
    def __abs__(self):
        return math.hypot(*self)
    
    def angle(self):
        return math.atan2(self.y ,self.x)
    
    def to_polar(self):
        return abs(self), self.angle()
    
    @classmethod
    def from_polar(cls, radius, angle):
        x = radius * math.cos(angle)
        y = radius * math.sin(angle)
        return Point(x, y)
    
    def __complex__(self):
        return complex(*self)

In [139]:
complex(Point(1,1))

(1+1j)

We might want to make our `Point` immutable. This is easy enough, we just need to change our `x` and `y` to `_x` and `_y` to convey that they should not be used directly, and add `x` and `y` properties. We shouldn't implement `__iadd__`, though, in this case.

In [14]:
import math

class Point:
    """A point (x,y) in the Euclidian plane.
    """
    
    def __init__(self, x, y):        
        self._x = float(x)
        self._y = float(y)
    
    @property
    def x(self):
        return self._x
    
    @property
    def y(self):
        return self._y
    
    def __iter__(self):
        yield self.x
        yield self.y
    
    def __repr__(self):
        return "{}({!r}, {!r})".format(type(self).__name__, *self)
    
    def __add__(self, other):
        if isinstance(other, numbers.Real):
            other = Point(other, other)
        if isinstance(other, type(self)):
            return Point(*(z1 + z2 for z1,z2 in zip(self, other)))
        return NotImplemented
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __gt__(self, other):
        return all(z1 > z2 for z1,z2 in zip(self, other))
    
    def is_extreme(self, *points):
        return all(self > point for point in points)
    
    def __abs__(self):
        return math.hypot(*self)
    
    def angle(self):
        return math.atan2(self.y ,self.x)
    
    def to_polar(self):
        return abs(self), self.angle()
    
    @classmethod
    def from_polar(cls, radius, angle):
        x = radius * math.cos(angle)
        y = radius * math.sin(angle)
        return Point(x, y)
    
    def __complex__(self):
        return complex(*self)

In [3]:
p = Point(1, 1)
print(p)
print(p + p)
print(p.to_polar())

Point(1.0, 1.0)
Point(2.0, 2.0)
(1.4142135623730951, 0.7853981633974483)


Note that we didn't have to change any other method (except remove `__iadd__`) because the `@property` decorator turns the `x` and `y` methods to instance members.

However, they cannot be directly changed now, making our class immutable, at least by convention:

In [4]:
p.x = 2

AttributeError: can't set attribute

An object attributes are saved in a dictionary called `__dict__`:

In [5]:
p.__dict__

{'_x': 1.0, '_y': 1.0}

However, a dictionary has some overhead, and a more memory efficient way to save the attributes is done by defining the class attribute `__slots__`:

In [15]:
class RegularPoint:
    
    def __init__(self, x, y):        
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        yield self.x
        yield self.y
    
    def __repr__(self):
        return "{}({!r}, {!r})".format(type(self).__name__, *self)
    
class EfficientPoint:
    __slots__ = ('x', 'y')
    
    def __init__(self, x, y):        
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        yield self.x
        yield self.y
    
    def __repr__(self):
        return "{}({!r}, {!r})".format(type(self).__name__, *self)

We load [ipython_memory_usage](https://github.com/ianozsvald/ipython_memory_usage), a package that monitors memory usage inside the notebook.

Install from with `pip install git+https://github.com/ianozsvald/ipython_memory_usage.git`, and on Windows you will also n
`conda install psutil`.

In [3]:
import ipython_memory_usage.ipython_memory_usage as imu
imu.start_watching_memory()

In [3] used 0.1523 MiB RAM in 29.48s, peaked 0.00 MiB above current, total RAM usage 38.52 MiB


In [7]:
lst1 = [RegularPoint(0, 0) for _ in range(10000000)]

In [7] used 2165.7812 MiB RAM in 29.80s, peaked 0.00 MiB above current, total RAM usage 2205.70 MiB


In [8]:
lst2 = [EfficientPoint(0, 0) for _ in range(10000000)]

In [8] used 1088.5469 MiB RAM in 29.12s, peaked 0.00 MiB above current, total RAM usage 3294.25 MiB


So we see that for $10^7$ points, the `__slots__` class requires ~2-fold less memory.

We should now delete the references to release these GB of RAM with the `del` statement:

In [9]:
del lst1
del lst2

In [9] used -3246.8281 MiB RAM in 3.57s, peaked 3246.80 MiB above current, total RAM usage 47.42 MiB


And finally stop monitoring the memory:

In [10]:
imu.stop_watching_memory()

## `Rectangle` class

We will implement two classes for rectangles, and compare the two implementations.

### First implementation - two points

The first implementation defines a rectangle by its lower left and upper right vertices.

In [28]:
class Rectangle1:
    """Describe a parallel-axes rectangle by storing two points.
    
    Attributes
    ----------
    llv : Point
        lower left vertex
    urv : Point
        upper right vertex
    """
    
    def __init__(self, lower_left_vertex, upper_right_vertex):        
        self.llv = Point(*lower_left_vertex)
        self.urv = Point(*upper_right_vertex)
        assert self.llv < self.urv, "Lower left vertex should be lower than upper right vertex"
        
    def __repr__(self):
        return "{}({!r}, {!r})".format(type(self).__name__, self.llv, self.urv)
        
    def __str__(self):
        return "Rectangle with lower left {} and upper right {}".format(self.llv, self.urv)

    def dimensions(self):
        height = self.urv.y - self.llv.y
        width = self.urv.x - self.llv.x
        return height, width
    
    def area(self):
        height, width = self.dimensions()
        area = height * width
        return area
       
    def transpose(self):
        """Reflection with regard to the line passing through lower left vertex with angle 315 (-45) degrees
        """
        height, width = self.dimensions()
        urv = self.llv
        llv = Point(self.llv.x - height, self.llv.y - width)
        return type(self)(llv, urv)

In [29]:
rect = Rectangle1(Point(0, 0), Point(2,1))
print(rect)
print("Area:", rect.area())
print("Dimensions:", rect.dimensions())
t_rect = rect.transpose()
print("Transposed:", t_rect)

Rectangle with lower left Point(0.0, 0.0) and upper right Point(2.0, 1.0)
Area: 2.0
Dimensions: (1.0, 2.0)
Transposed: Rectangle with lower left Point(-1.0, -2.0) and upper right Point(0.0, 0.0)


## Exercise

Overload the `__contains__` method of the rectangle class so that given a point it will return `True` if the point is inside the rectangle and `False` otherwise.

In [84]:
rect = Rectangle2(Point(0, 0), 1, 2) # or use Rectangle1 if you prefer
assert Point(0.5, 0.5) in rect
assert Point(1.5, 0.5) in rect
assert Point(2.5, 0.5) not in rect
assert Point(0.5, 2.5) not in rect

## Exercise

The second implementation defines a rectangle by the lower left point, the height and the width.

Define the exact same methods as in `Rectangle1`, with the same input and output, but  different inner representation / implementation.

In [40]:
class Rectangle2:
    """Describe a parallel-axes rectangle by storing lower left point, height and width.
    
    Attributes
    ----------
    point : Point
        lower left point
    width : float
        width
    height : float
        height
    """
    pass

In [41]:
rect = Rectangle2(Point(0, 0), 1, 2)
print(rect)
print("Area:", rect.area())
print("Dimensions:", rect.dimensions())
t_rect = rect.transpose()
print("Transposed:", t_rect)

Rectangle with lower left Point(0.0, 0.0) and upper right Point(2.0, 1.0)
Area: 2.0
Dimensions: (1.0, 2.0)
Transposed: Rectangle with lower left Point(-1.0, -2.0) and upper right Point(0.0, 0.0)


# Inheritance

Next we will see how inheritance works in Python.

We define a `Door` class. The door is either open or closed, it can be opened or closed, and it can be represented as a string.

In [33]:
class Door:
    OPEN = 'open'
    CLOSED = 'closed'
    
    def __init__(self, status=CLOSED):
        self._status = status
        
    @property
    def status(self):
        return self._status

    @status.setter
    def status(self, new_status):
        if new_status not in (self.OPEN, self.CLOSED):
            raise ValueError('Status must be one of {}'.format((self.OPEN, self.CLOSED)))
        self._status = new_status
        return new_status
    
    def __repr__(self):
        return '{}(status={})'.format(type(self).__name__, self.status)

    def __str__(self):
        return 'Door {0} is {1}'.format(id(self), self.status)
    
    def open(self):
        self.status = self.OPEN
        
    def close(self):
        self.status = self.CLOSED

In [35]:
door = Door()
print(door)
door

Door 74438080 is closed


Door(status=closed)

Now we want to define a secure door which only opens with a password.

The secure door inherits from the regular door, but it makes some changes:

- `__init__` accepts a password and saves it's hash as an attribute
- `open` requires the password from the user to actually open the door

In [38]:
from getpass import getpass
from hashlib import sha224

In [39]:
class SecurityDoor(Door):
    """A door that requires a password to open.
    """
  
    def __init__(self, password):
        super(SecurityDoor, self).__init__()
        self.secret = self.digest(password)
        
    def open(self):
        if self.digest(getpass("What is the password?")) == self.secret:
            super(SecurityDoor, self).open()
        else:
            print("Wrong password!")
            
    def digest(self, password):
        """Hash a password using sha224 algorithm.
        """
        return sha224(password.encode('utf8')).hexdigest()

In [40]:
secure_door = SecurityDoor('opensesame')
secure_door.open()
print(secure_door)
secure_door.close()
print(secure_door)

What is the password?········
Door 78324400 is open
Door 78324400 is closed


Multiple inheritance works in Python. If there are clashes between names Python will search from the first parent to the last. 

However, multiple inheritance and inheritance in general are many times not neccessary and similar results can be produced by composition and delegation, without creating the complexity and constraints that inheritance induces.

In the words of the famous [Design Patterns](https://en.wikipedia.org/wiki/Composition_over_inheritance) book:

> Favor 'object composition' over 'class inheritance'.

# Composition and delegation

Composition means that one object **explicitly delegates** some tasks to another object. 

Delegation is implicit in inheritance, and explicit in composition.
The *Zen of Python* (`import this`) says that *Explicit is better than implicit*, so it we should conclude that composition is better than inheritance.

Let's implement regular composition, which simply makes an object part of the other as an attribute:

In [46]:
class SecurityDoor:
    """A door that requires a password to open.
    """
    
    def __init__(self, password):
        self._door = Door(status=Door.CLOSED)
        self._secret = self.digest(password)
    
    @property
    def secret(self):
        return self._secret
    
    @property
    def door(self):
        return self._door
    
    def __repr__(self):
        return repr(self.door)
               
    def open(self):
        if self.digest(getpass("What is the password?")) == self.secret:
            self.door.open()
        else:
            print("Wrong password!")
        
    def close(self):
        self.door.close()
        
    def digest(self, password):
        """Hash a password using sha224 algorithm.
        """
        return sha224(password.encode('utf8')).hexdigest()

In [47]:
secure_door = SecurityDoor('opensesame')
secure_door.open()
print(secure_door)
secure_door.close()
print(secure_door)

What is the password?········
Door(status=open)
Door(status=closed)




The primary goal of composition is to relax the coupling between objects. 
This little example shows that now `SecurityDoor` is an `object` and no more a `Door`, 
which means that the internal structure of `Door` is not copied. 
For this very simple example both `Door` and `SecurityDoor` are not big classes, 
but in a real system objects can very complex; 
this means that their allocation consumes a lot of memory and if a system contains thousands or millions of objects that could be an issue.

The composed `SecurityDoor` has to redefine every attribute since the concept of delegation applies only to methods and not to attributes, doesn't it?

**No.** Python allows objects manipulation and attribute access is one of the most useful. 
Accessing attributes is ruled by a special method called `__getattribute__` that is called whenever an attribute of the object is accessed. 
Overriding `__getattribute__`, however, is overkill; 
it is a very complex method, and, being called on every attribute access, any change makes the whole thing slower.

The method we have to leverage to delegate attribute access is `__getattr__`, which is a special method that is called whenever the requested attribute is not found in the object. 
So basically it is the right place to dispatch all attribute and method access our object cannot handle. 

The previous example becomes:

In [48]:
class SecurityDoor:
    """A door that requires a password to open.
    """
    
    def __init__(self, password):
        self._door = Door(status=Door.CLOSED)
        self._secret = self.digest(password)
              
    @property
    def door(self):
        return self._door
    
    @property
    def secret(self):
        return self._secret
            
    def __repr__(self):
        return repr(self.door)
    
    def open(self):
        if self.digest(getpass("What is the password?")) == self.secret:
            self.door.open()
        else:
            print("Wrong password!")
        
    def __getattr__(self, attr):
        return getattr(self.door, attr)
    
    def digest(self, password):
        """Hash a password using sha224 algorithm.
        """
        return sha224(password.encode('utf8')).hexdigest()

In [49]:
secure_door = SecurityDoor('opensesame')
secure_door.open()
print(secure_door)
secure_door.close()
print(secure_door)

What is the password?········
Door(status=open)
Door(status=closed)


As this last example shows, delegating every member access through `__getattr__` is very simple. 
Pay attention to `getattr` which is different from `__getattr__`. 
The former is a built-in function that is equivalent to the dotted syntax, i.e. `getattr(obj, 'someattr')` is the same as `obj.someattr`, but you have to use it since the name of the attribute is contained in a string.

**Composition provides a superior way to manage delegation** since it can selectively delegate the access, even mask some attributes or methods, while inheritance cannot. 

In Python you also avoid the memory problems that might arise with delegation when you put many objects inside another; Python handles everything through its reference, so the size of an attribute is constant and very limited.

## Exercise

Define a new class, `RevolvingDoor`, which closes immediately after it is opened. Define it once with inheritance and once with composition.

In [ ]:
class RevolvingDoor(Door):
    pass

rdoor = RevolvingDoor()
rdoor.open()
print(rdoor)

In [ ]:
class RevolvingDoor:
    pass
    
rdoor = RevolvingDoor()
rdoor.open()
print(rdoor)

# Polymorphism

In Python, polymorphism is baked into the language, due to the **Duck typing** principle. We saw above how this relates to methods like `__add__`, `__sub__`, `__repr__`, and `__contains__`.

Another example is a file object:

In [51]:
f = open("../data/crops.txt")
print(type(f))
print(dir(f))
f.close()

<class '_io.TextIOWrapper'>
['_CHUNK_SIZE', '__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__iter__', '__le__', '__lt__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_checkClosed', '_checkReadable', '_checkSeekable', '_checkWritable', '_finalizing', 'buffer', 'close', 'closed', 'detach', 'encoding', 'errors', 'fileno', 'flush', 'isatty', 'line_buffering', 'mode', 'name', 'newlines', 'read', 'readable', 'readline', 'readlines', 'seek', 'seekable', 'tell', 'truncate', 'writable', 'write', 'writelines']


Say we write a function that gets a file and returns all the lines that start with a given prefix (say, 'Am'):

In [20]:
def filter_by_prefix(file, prefix):
    return [line.strip() for line in file if line.startswith(prefix)]

with open('../data/crops.txt', 'r') as f:
    print(filter_by_prefix(f, 'Am'))

['Amaranthus cruentus', 'Amaranthus spp.', 'Amaranthus viridis', 'Amomum subulatum', 'Amorphophallus konjac', 'Amorphophallus paeoniifolius']


But now, say we want to read from `crops.txt.gz`, which is compressed with gzip. There is a module for reading gzipped files:

In [53]:
import gzip

In [54]:
gzfile = gzip.open(r'..\data\crops.txt.gz' ,'r')
print(type(gzfile))
print(dir(gzfile))
print(isinstance(gzfile, f.__class__))
gzfile.close()

<class 'gzip.GzipFile'>
['__abstractmethods__', '__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_abc_cache', '_abc_negative_cache', '_abc_negative_cache_version', '_abc_registry', '_buffer', '_checkClosed', '_checkReadable', '_checkSeekable', '_checkWritable', '_check_can_read', '_check_can_seek', '_check_can_write', '_check_not_closed', '_init_write', '_write_gzip_header', 'close', 'closed', 'detach', 'filename', 'fileno', 'fileobj', 'flush', 'isatty', 'mode', 'mtime', 'myfileobj', 'name', 'peek', 'read', 'read1', 'readable', 'readinto', 'readinto1', 'readline', 'readlines', 'rewind', 'seek', 'seekable', 'tell', 'truncate', 'writable', 'write', 'writelines']
False


You notice that this is a different type than a file we opened with `open` (that would be `f`), and the is not even inheritance relationship (we know this because `isinstace` returned `False`). 

But our function `filter_by_prefix` doesn't care about the type, all it wants is an **object that you can loop over with a `for`**: an iterable object that implements the `__iter__` method by either:

- returning an iterator, which is an object that has a `next` method and stops iteration by raising a `StopIteration` exception, or 
- using the `yield` statement, which creates a generator.

Indeed, `gzfile` implements `__iter__`:

In [59]:
hasattr(gzfile, '__iter__')

True

So we can use it with our function:

In [56]:
with gzip.open('../data/crops.txt.gz', 'rt') as gzfile: # rt is for reading text
    print(filter_by_prefix(gzfile, 'Am'))

['Amaranthus cruentus', 'Amaranthus spp.', 'Amaranthus viridis', 'Amomum subulatum', 'Amorphophallus konjac', 'Amorphophallus paeoniifolius']


It doesn't even have to be a file. A list is just as good:

In [55]:
continents = ['America', 'Europe', 'Asia', 'Africa', 'Anarctica', 'Australia']
print(filter_by_prefix(continents, 'Am'))

['America']


# Destructor

Python uses a garbase collector, and the standard CPython implementation uses reference counting and deletes objects when there are no more references to them.

You can remove references using the `del` command (see in [Python Tutor](http://pythontutor.com/visualize.html#code=a+%3D+%5B1,2,3%5D%0Adel+a&mode=display&origin=opt-frontend.js&cumulative=false&heapPrimitives=false&textReferences=false&py=3&rawInputLstJSON=%5B%5D&curInstr=2)):

In [1]:
a = [1,2,3]
del a
a

NameError: name 'a' is not defined

a = [1,2,3]
del a

You can implement the [`__del__`](https://docs.python.org/3.5/reference/datamodel.html#object.__del__) method which is called when an instance is about to be destroyed. 
This is also called a **destructor**. 
Similar to `__init__`, if a base (parent) class has a `__del__` method then you must explicitly call it. 
It is not guaranteed that __del__() methods are called for objects that still exist when the interpreter exits.

In [12]:
class A:
    def __init__(self, name):
        self.name = name
        print('Initialize')
        
    def __del__(self):
        print('Delete')
        
    def __repr__(self):
        return self.name

In [27]:
a = A('Alex')
print(a)

Initialize
Alex


In [28]:
b = a
print(b)

Alex


In [29]:
del a
print(b)

Alex


In [30]:
del b

Delete


Note that several things can implicitly save a reference to your object, including the `_` variable name and stack traces.

# References

- Python 3 [OOP tutorial](https://docs.python.org/3/tutorial/classes.html)
- Leonardo Giordani's [OOP notebooks](http://nbviewer.jupyter.org/github/lgiordani/blog_source/blob/master/pelican/content/notebooks/Python_3_OOP_Part_3__Delegation__composition_and_inheritance.ipynb)
- [ Fluent Python](http://shop.oreilly.com/product/0636920032519.do) by Luciano Ramalho is a great book for intermediate and advanced Python programmers.
- [Python data model docs](https://docs.python.org/3.5/reference/datamodel.html).

## Colophon
This notebook was written by [Yoav Ram](http://www.yoavram.com) and is part of the _Python for Engineers_ course.

The notebook was written using [Python](http://pytho.org/) 3.4.4, [IPython](http://ipython.org/) 4.0.3 and [Jupyter](http://jupyter.org) 4.0.6.

This work is licensed under a CC BY-NC-SA 4.0 International License.

![Python logo](https://www.python.org/static/community_logos/python-logo.png)